In [42]:
import pandas as pd

In [43]:
df = pd.read_csv("C:\\MyWork\\MyLearning\\ML\\Files\\DataSet\\SPAM text message 20170820 - Data.csv")

In [44]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [45]:
# Convert Categorical output

df['Category'] = [1 if x == 'spam' else 0 for x in df['Category']]

df['Message'] = [(re.sub(r"[^a-zA-Z /s]",'',x).lower()) for x in df['Message']] # Remove 

In [46]:
df.head()

,Category,Message
0,0,go until jurong point crazy available only in ...
1,0,ok lar joking wif u oni
2,1,free entry in a wkly comp to win fa cup final...
3,0,u dun say so early hor u c already then say
4,0,nah i dont think he goes to usf he lives aroun...


In [60]:
from nltk.tokenize import word_tokenize

df['NL_words'] = [word_tokenize(x) for x in df['Message']]

In [64]:
from nltk.stem import WordNetLemmatizer

['go',
 'until',
 'jurong',
 'point',
 'crazy',
 'available',
 'only',
 'in',
 'bugis',
 'n',
 'great',
 'world',
 'la',
 'e',
 'buffet',
 'cine',
 'there',
 'got',
 'amore',
 'wat']

In [67]:
df['NL_words'] = [[WordNetLemmatizer().lemmatize(y) for y in x] for x in df['NL_words']]

In [74]:
df['Message'] = [' '.join([y for y in x]) for x in df['NL_words']]

In [131]:
from sklearn.feature_extraction.text import CountVectorizer
max_features = 3000 #We use the most common word
count_vectorizer = CountVectorizer(max_features = max_features, stop_words = "english")
sparce_matrix = count_vectorizer.fit_transform(df['Message']).toarray()


In [123]:
print("the most using {} words: {}".format(max_features,count_vectorizer.get_feature_names()))

the most using 3000 words: ['aah', 'aathilove', 'aathiwhere', 'abi', 'ability', 'abiola', 'abj', 'able', 'abt', 'abta', 'aburo', 'ac', 'academic', 'acc', 'accept', 'access', 'accident', 'accidentally', 'accordingly', 'account', 'ache', 'aclpm', 'acoentry', 'acted', 'acting', 'action', 'activate', 'active', 'activity', 'actor', 'actual', 'actually', 'ad', 'adam', 'add', 'addamsfa', 'added', 'addicted', 'addie', 'address', 'admin', 'administrator', 'admirer', 'adore', 'adoring', 'adult', 'advance', 'adventure', 'advice', 'advise', 'ae', 'aeronautics', 'aeroplane', 'affair', 'affection', 'affectionate', 'afraid', 'aft', 'afternoon', 'aftr', 'ag', 'agalla', 'age', 'agent', 'ago', 'agree', 'ah', 'aha', 'ahead', 'ahmad', 'ahsen', 'aid', 'aight', 'aint', 'air', 'airport', 'airtel', 'aiya', 'aiyah', 'aiyar', 'aiyo', 'aj', 'aka', 'al', 'alaipayuthe', 'album', 'alcohol', 'alert', 'alex', 'alfie', 'algarve', 'alive', 'allah', 'allow', 'allowed', 'alright', 'alrite', 'alwys', 'amazing', 'american'

In [89]:
y = df.iloc[:,0].values   # male or female classes

In [91]:
x = sparce_matrix

In [92]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.1, random_state = 42)

In [93]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(x_train,y_train)
print("the accuracy of our model: {}".format(nb.score(x_test,y_test)))

the accuracy of our model: 0.8691756272401434


In [208]:
msg1 = df['Message'][9]
category1 = df['Category'][9]

print(msg1)
print("Expected :",category1)

import numpy as np

msg1_vector = count_vectorizer.transform([msg1]).toarray()

print("Actual:",nb.predict(msg1_vector))

had your mobile month or more u r entitled to update to the latest colour mobile with camera for free call the mobile update co free on
Expected : 1
Actual: [1]


In [213]:
msg2 = df['Message'][10]
category2 = df['Category'][10]

print(msg2)
print("Expected :",category2)

msg2_vector = count_vectorizer.transform([msg2]).toarray()

print("Actual:",nb.predict(msg2_vector))


im gon na be home soon and i dont want to talk about this stuff anymore tonight k ive cried enough today
Expected : 0
Actual: [0]


In [94]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter = 200)
lr.fit(x_train,y_train)
print("our accuracy is: {}".format(lr.score(x_test,y_test)))

c:\users\insakum46\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


our accuracy is: 0.9802867383512545


In [102]:
x_train.shape

(5014, 3000)

In [103]:
x_test.shape

(558, 3000)

In [104]:
# Using Deep learning Keras

x_test = x_test.reshape(558,3000,1)
x_train = x_train.reshape(5014,3000,1)

In [109]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout

model = Sequential()
model.add(LSTM(50,return_sequences = True, input_shape = (x_train.shape[1], 1)))
model.add(Dropout(0.2))

model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50))
model.add(Dropout(0.2))

# Adding the output layer
model.add(Dense(units = 1))

In [111]:
# Compiling the RNN
model.compile(optimizer = 'adam', loss = 'mean_squared_error',metrics=["accuracy"])

In [113]:
# Fitting the RNN to the Training set
model.fit(x_test, y_test, epochs = 3, batch_size = 32)



Epoch 1/3
558/558 [==============================] - 309s 554ms/step - loss: 0.1190 - accuracy: 0.8692
Epoch 2/3
558/558 [==============================] - 353s 632ms/step - loss: 0.1147 - accuracy: 0.8692
Epoch 3/3
558/558 [==============================] - 333s 597ms/step - loss: 0.1146 - accuracy: 0.8692


In [220]:
msg1_vector=msg1_vector.reshape(1,3000,1) 

In [221]:
model.predict(msg1_vector)

array([[0.125506]], dtype=float32)